## Environment setup

Prepare `pytorch-1.5.1`, `cuda-10.1`, and `cudnn-v7.6.5`

In [1]:
!nvidia-smi

Sat Apr  9 05:53:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 5000     Off  | 00000000:00:05.0 Off |                  Off |
| 33%   28C    P8     8W / 230W |      0MiB / 16125MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone --recursive git@github.com:ScrapPaper/off-belief-small.git

Cloning into 'off-belief-small'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 242 (delta 69), reused 235 (delta 62), pack-reused 0
Receiving objects: 100% (242/242), 15.10 MiB | 15.80 MiB/s, done.
Resolving deltas: 100% (69/69), done.
Submodule 'third_party/pybind11' (https://github.com/pybind/pybind11.git) registered for path 'third_party/pybind11'
Cloning into '/notebooks/off-belief-small/third_party/pybind11'...
remote: Enumerating objects: 23417, done.        
remote: Counting objects: 100% (1292/1292), done.        
remote: Compressing objects: 100% (505/505), done.        
remote: Total 23417 (delta 808), reused 1053 (delta 716), pack-reused 22125        
Receiving objects: 100% (23417/23417), 8.56 MiB | 12.98 MiB/s, done.
Resolving deltas: 100% (16232/16232), done.
Submodule path 'third_party/pybind11': checked out 'a1b71df137e015d44f7e31f7b6d4807253fb7871'
Submodule 'to

In [ ]:
!nvcc --version # cuda-10.1
!cat /usr/include/cudnn.h | grep CUDNN_MAJOR -A 2 # cudnn-v7.6.5
!ls -al /usr/local/cuda-10.0/compat/libcuda.so.410.104

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
#define CUDNN_MAJOR 7
#define CUDNN_MINOR 6
#define CUDNN_PATCHLEVEL 2
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#include "driver_types.h"
-rw-r--r-- 1 root root 15672664 Feb  6  2019 /usr/local/cuda-10.0/compat/libcuda.so.410.104


In [1]:
!ls -al /usr/lib/x86_64-linux-gnu/libcuda.so.*

lrwxrwxrwx 1 root root       20 Nov 24  2020 /usr/lib/x86_64-linux-gnu/libcuda.so.1 -> libcuda.so.450.36.06
-rw-r--r-- 1 root root 15672664 Feb  6  2019 /usr/lib/x86_64-linux-gnu/libcuda.so.410.104
-rw-r--r-- 1 root root 18948104 Jun  1  2020 /usr/lib/x86_64-linux-gnu/libcuda.so.450.36.06


In [2]:
!apt update
!apt install -y build-essential libpq-dev libssl-dev openssl libffi-dev
!apt install -y python3-pip python3.7-dev
!apt install python3.7
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 1
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 2
!update-alternatives --set python3 /usr/bin/python3.7
!ln -sf /usr/bin/python3 /usr/local/bin/python
!ln -sf /usr/bin/python3 /usr/local/bin/python3
!python -m pip install --upgrade pip
!python -m pip install cython
!python -m pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html --progress-bar off
!python -m pip install psutil cmake==3.15.3

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]               
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1498 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [909 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2734 kB]33m
Get:7 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [21.1 kB]3m
Get:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 Packages [45.3 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]0m 
Get:11 http://archive.ubuntu.com/ubuntu bionic/main amd64 Packages [1344 kB]   
Get:12 http://archive.ubuntu.com/ubuntu bionic/restricted amd64 Packag

In [8]:
# avoid tensor operation using all cpu cores
%env OMP_NUM_THREADS 1

env: OMP_NUM_THREADS=1


## Enable observation logging

* Incorporate basic observation logging functionality to the `verbose` option in `HanabiEnv` 

In [3]:
%cd /notebooks/off-belief-small/

/notebooks/off-belief-small


In [4]:
# insert logging code
!cp -n rlcc/hanabi_env.h rlcc/hanabi_env.h.old
!sed -r -e 's/^( *numStep_ = 0;)$/\1\n    if (verbose_) std::cout << "STATE\\nCurrent player: " << state_->CurPlayer() << "\\n" << state_->ToString() << "\\n\\n";/' \\
        -e 's/^(.*applyMove.*)$/    if (verbose_) std::cout << "MOVE: " << move.ToString() << "\\n\\n";\n\1\n    if (verbose_) std::cout << "STATE\\nCurrent player: " << state_->CurPlayer() << "\\n" << state_->ToString() << "\\n\\n";/g' \\
        rlcc/hanabi_env.h.old > rlcc/hanabi_env.h

sed: can't read  : No such file or directory
sed: can't read  : No such file or directory


In [5]:
# hide card ID output (std::to_string(Id()) + " ")
!cp -n hanabi-learning-environment/hanabi_lib/hanabi_card.cc hanabi-learning-environment/hanabi_lib/hanabi_card.cc.old
!sed -r 's/std.*" "/std::string\(\)/g' \\
        hanabi-learning-environment/hanabi_lib/hanabi_card.cc.old > hanabi-learning-environment/hanabi_lib/hanabi_card.cc

sed: can't read  : No such file or directory


## Compile learning environment

In [6]:
!rm -rf build
!mkdir build
%cd build
!cmake ..
!make -j10

/notebooks/off-belief-small/build
-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Traceback (most recent call last):
  File "/notebooks/off-belief-small/get_pybind_flags.py", line 5, in <module>
    val = getattr(torch._C, f"_PYBIND11_{name}")
AttributeError: module 'torch._C' has no attribute '_PYBIND11_COMPILER_TYPE'
-- Found PythonInterp: /usr/bin/python3.7 (found suitable version "3.7.13", minimum required is "3.7") 
-- Found PythonLibs: /usr/lib

## Generate observation log

* Evaluate the agent in Hanabi games with the `verbose` option to produce a log

In [ ]:
# extract log from 'start' to 'end' games
def extract_log(log_path, start, end, out_path):
    with open(log_path) as log:
        with open(out_path, 'w') as out:
            game = -1
            for line in log:
                if line.startswith('Hanabi game created'):
                    game += 1
                    if end <= game:
                        break
                if start <= game:
                    out.write(line)

In [ ]:
%cd ../pyhanabi
!sed -i -r "s/^( *)False/\1True/g" create.py # enable verbose logging

/notebooks/off-belief-small/pyhanabi


### Hanabi-Small

* **States per game**: ~31
* **Sampled states per game**: 4 (~13%)
* **Train set**: 25,000 games (100,000 states)
* **Test set**: 5,000 games (20,000 states)

In [ ]:
!python tools/eval_model.py --weight1 exps/obl1/model0.pthw --num_game 1 --num_run 30000 > small.log

In [ ]:
extract_log('small.log',     0, 25000, 'small_train.log')
extract_log('small.log', 25000, 30000, 'small_test.log')

### Standard Hanabi

* **States per game**: ~69
* **Sampled states per game**: 7 (~10%)
* **Train set**: 15,000 games (105,000 states)
* **Test set**: 3,000 games (21,000 states)

In [ ]:
!python tools/eval_model.py --paper obl --num_game 1 --num_run 18000 > standard.log

In [ ]:
extract_log('standard.log',     0, 15000, 'train.log')
extract_log('standard.log', 15000, 18000, 'test.log')

## Parse logs to datasets

* An example of an observation is shown below:

  ```
  ...
  MOVE: (Reveal player +1 color R)
  
  STATE
  Current player: 1
  Life tokens: 1
  Info tokens: 1
  Fireworks: R3 Y0
  Hands:
  Y2 || X2|RY2
  Y1 || XX|RY12345
  -----
  Cur player
  Y1 || X1|Y1
  R2 || RX|R12345
  Deck size: 8
  Discards: R3 R5 Y2 R1 Y3
  
  MOVE: (Play 0)
  ...
  ```
  
* **An observation of the above state from the perspective of `Cur player` contains the following information**:
  
  * **Identities of their partner's cards** (i.e., their partner's first (oldest) card is `Y2`:
    
    $\lnot is(R, c_1)\land is(Y, c_1)\land \lnot is(1, c_1)\land is(2, c_1)\land \lnot is(3\ldots5, c_1)$
    
  * **Plausible identities of their own cards from previously obtained clues** (i.e., their second (newest) card can be `R12345`:
    
    $ps(R, c_2)\land \lnot ps(Y, c_2)\land ps(1\ldots 5, c_2)$
    
  * **State of the firework stacks**:
    
    $\ fw(R)=3\land fw(Y)=0$
    
  * **Number of discarded cards**:
    
    $ds(R,1)=1\land ds(R,2)=0\land ds(R,3)=1\land ds(R,4)=0\land ds(R,5)=1\land ds(Y,1)=0\land\ldots\land ds(Y,5)=0$
    
  * **The most recent action** (i.e., `Reveal player +1 color R`):
    
    $\ COLOUR\land\lnot RANK\land\lnot DISCARD \land\lnot PLAY$

* `log_to_csv.py` can extract such information from observation logs to form instances in oracle and test datasets
* Atomic formulae (e.g. $\lnot is(R, c_1)$) are translated into propositional variables (e.g. $\lnot isR\_c1$)
* The instances are labelled with the observed action of the agent (e.g. `(Play 0)`), converted to 1-based card indexing (e.g., `PLAY_1`)

In [ ]:
!python3 log_to_csv.py train.csv < train.log
!python3 log_to_csv.py test.csv  < test.log

In [ ]:
!sed -r -e "s/^(HAND_SIZE =).*/\1 2/g" \\
        -e "s/^(N_COLOURS =).*/\1 2/g" \\
        -e "s/^(N_RANKS =).*/\1 5/g" \\
        -e "s/^(N_LIVES =).*/\1 1/g" \\
        -e "s/^(N_SAMPLES =).*/\1 4/g" \\
        log_to_csv.py > log_to_csv_small.py
!python3 log_to_csv_small.py small_train.csv < small_train.log
!python3 log_to_csv_small.py small_test.csv  < small_test.log